<h1>Final Year Project - The Third Eye </h1>
Done by: See Zhuo Yi Joey (2011927), Liu Zhen (2021250), Koh Hui Lyn (2021672) and Ang Jun Hoa (2040295)
<br/>Project Aim: Using computer vision to aid coach’s analysis of a bowler’s performance by producing consistent and accurate intelligent analysis.
<br/>Modules Required: MediaPipe, OpenCV


<h2>Installing Required Packages</h2>

In [1]:
# pip install mediapipe
# pip install opencv-python

<h2>Importing</h2>

In [2]:
# Importing OpenCV to help us process (read/write) videos
import cv2

# Import Math to help us with some calculations
import math as m

# Import pandas to help us with storing of previous frames' information
import pandas as pd

# Import MediaPipe to help us with Pose Estimation
import mediapipe as mp

# Import some utils modules
import os
import datetime
import time

<h2>Functions</h2>

In [3]:
# Find Distance between 2 points using Pythagoras Theorem
def findDistance(x1, y1, x2, y2):
    dist = m.sqrt((x2-x1)**2+(y2-y1)**2)
    return dist

# Calculate angle between 2 points
def findAngle(x1, y1, x2, y2):
    theta = m.acos((y2 -y1)*(-y1) / (m.sqrt((x2 - x1)**2 + (y2 - y1)**2) * y1))
    degree = int(180/m.pi)*theta
    return degree

# Calculate difference of X-Coordinate of two points
def findX(x_knee,x_hand):
  X = x_hand - x_knee
  return X


<h2>Utils</h2>

In [4]:
# Font (For OpenCV Video)
font = cv2.FONT_HERSHEY_SIMPLEX

# Preset Colors for easy calling
blue = (255, 127, 0)
red = (50, 50, 255)
green = (127, 255, 0)
dark_blue = (127, 20, 0)
light_green = (127, 233, 100)
yellow = (0, 255, 255)
pink = (255, 0, 255)

In [5]:
# From all the Mediapipe Computer Vision Solutions, select to use Mediapipe Pose============================================
mp_pose = mp.solutions.pose

# Call the MediaPipe Pose Model with defined parameters.
# min_detection_confidence - minimum confidence required to detect a PERSON (not landmarks)
# min_tracking_confidence  - minimum confidence required to detect the landmarks
# model_complexity - complexity of the pose landmark model (0,1,2) Where 2 is the most complex, increasing landmark accuracy and time taken to run
# smooth_landmarks - reduce the jitter for the detected landmarks based on the previous landmark position
pose = mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.8, model_complexity=2, smooth_landmarks=True)


# Choose which video to use=================================================================================================
file_name = './videos/coachVids3/20220725_205854.mp4'
cap = cv2.VideoCapture(file_name)

# CV2  properties===========================================================================================================

# Get the FPS, Width, and Height of the Video
fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Getting the frame size ie 1920 x 1080
frame_size = (width, height)

# Video Codec to help store and playback the video (required for the VideoWriter)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')

# Initialize video writer with the output filename, fourcc, fps of the vid and frame size
video_output = cv2.VideoWriter("20220725_205854.mp4", fourcc, fps, frame_size)

<h2>Main Code</h2>

In [6]:


# Preparing the dataframes

# Dataframe to store the Velocity Information
feetVelo = pd.DataFrame(columns=["Frame","LH_X","LI_X", "Heel Velo", "Index Velo", "Wrist_Y","Wrist_X"])

# Dataframe to get the average FeetSize
feetSizeInfo = pd.DataFrame(columns=['Frame', 'AvgLen'])



In [7]:
# Start of the program=======================================================================

print('Starting...')

# Variables for dynamic font size
fontsize = 1
thick = 4
text_y = 50
font_access = 1

# Variables for Step Counter
steps = 0
stage = None
maxFeetLength = 50

# Variables for Timing
access = 1
velo = 0
ball_release = None
sliding = False
kneesDis = 0
diffBtY = 0
diffBtX = 0
throwing = False

currentFrame= 0
currentframenumber=dict.fromkeys(["Angle1","Angle2","Angle3","Angle4","Angle5","Release"])
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

while cap.isOpened():
    # Capture frames
    success, image = cap.read()
    if not success:
        print("No frames left to process!!!")
        break
    # Get fps, height and width
    fps = cap.get(cv2.CAP_PROP_FPS)
    h, w = image.shape[:2]
    # Convert the BGR image to RGB
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    # Process the frame with Mediapipe Pose
    keypoints = pose.process(image)
    # Convert the image back to BGR
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    # Get the current frame number
    currentFrame = cap.get(cv2.CAP_PROP_POS_FRAMES)

    
    #============ Getting landmarks ============
    lm = keypoints.pose_landmarks
    lmPose = mp_pose.PoseLandmark
    # If Landmarks are able to be detected
    if lm is not None:
        # To account to most videos being rectangular, we have to multiply width and height
        # X-Axis will multiply be the width of the video and the Y-Axis will multiply by the height
        # This is to remain the aspect ratio of the video, and make sure the X and Y will be in the same scale.

        # For Angle Component 
        l_shldr_x = int(lm.landmark[lmPose.LEFT_SHOULDER].x * w)
        l_shldr_y = int(lm.landmark[lmPose.LEFT_SHOULDER].y * h)
        r_shldr_x = int(lm.landmark[lmPose.RIGHT_SHOULDER].x * w)
        r_shldr_y = int(lm.landmark[lmPose.RIGHT_SHOULDER].y * h)
        l_hip_x = int(lm.landmark[lmPose.LEFT_HIP].x * w)
        l_hip_y = int(lm.landmark[lmPose.LEFT_HIP].y * h)

        # Ankles for feet distance calculation
        l_ank_x = int(lm.landmark[lmPose.LEFT_ANKLE].x * w)
        l_ank_y = int(lm.landmark[lmPose.LEFT_ANKLE].y * h)
        r_ank_x = int(lm.landmark[lmPose.RIGHT_ANKLE].x * w)
        r_ank_y = int(lm.landmark[lmPose.RIGHT_ANKLE].y * h)

        r_ind_x = int(lm.landmark[lmPose.RIGHT_INDEX].x * w)
        r_ind_y = int(lm.landmark[lmPose.RIGHT_INDEX].y * h)
        r_heel_x = int(lm.landmark[lmPose.RIGHT_HEEL].x * w)
        r_heel_y = int(lm.landmark[lmPose.RIGHT_HEEL].y * h)

        l_ind_x = int(lm.landmark[lmPose.LEFT_INDEX].x * w)
        l_ind_y = int(lm.landmark[lmPose.LEFT_INDEX].y * h)
        l_heel_x = int(lm.landmark[lmPose.LEFT_HEEL].x * w)
        l_heel_y = int(lm.landmark[lmPose.LEFT_HEEL].y * h)

        # For Timing Component
        r_wrist_x = int(lm.landmark[lmPose.RIGHT_WRIST].x * w)
        r_knee_x = int(lm.landmark[lmPose.RIGHT_KNEE].x * w)
        l_knee_x = int(lm.landmark[lmPose.LEFT_KNEE].x * w)
        r_wrist_y = int(lm.landmark[lmPose.RIGHT_WRIST].y * w)


        # When x-val is at the max, foot is placed on the floor
        if (currentFrame < 20):
            feetLength = r_ind_x - r_heel_x
            if feetLength > maxFeetLength:
                maxFeetLength = feetLength



        #============ Functions ============
        # feetDist = findDistance(l_ank_x, l_ank_y, r_ank_x, r_ank_y)
        feetDist = abs(findX(l_ank_x, r_ank_x))

        # Steps Counter (To be improved - ie Thresholds improvements)
        if steps < 5:
            if feetDist > maxFeetLength and stage == 'up':
                steps += 1
                stage = "down"
            elif feetDist < maxFeetLength:
                stage = "up"
        # Timing Component
        # When feet dist is big, then it means it's sliding. Sooooooo enter this

        if currentFrame > 20:
            preFrame_Y = currentFrame - 3
            diffBtY = r_wrist_y - feetVelo["Wrist_Y"][preFrame_Y]
            diffBtX = r_wrist_x - feetVelo["Wrist_X"][preFrame_Y]
            if diffBtY > 0 and r_shldr_x>r_wrist_x:
                throwing = True
            else:
                throwing = False

        # if (feetDist > 3*maxFeetLength and steps ==5)or sliding==True:
        if (feetDist > 3*maxFeetLength and throwing == True) or sliding==True :
            sliding=True
            # currentFrame = cap.get(cv2.CAP_PROP_POS_FRAMES)
            pre = currentFrame - 2
            if currentFrame > 10:
                # Calculate Velocity with this frame and 4 frames before
                velo = abs(((l_heel_x - feetVelo["LH_X"][pre])/(currentFrame-pre)))
                if velo < 1.5 and access == 1:
                    kneesDis = abs(r_knee_x - l_knee_x)
                    print("distance between knee are ", kneesDis)
                    access = 0
                    ball_train_feet_dis = findX(r_knee_x, r_wrist_x)
                    ball_slide_feet_dis = findX(l_knee_x, r_wrist_x)
                    # Calculate distance between hand and knees to determine ball release type
                    if ball_train_feet_dis < 0:
                        ballDisBtKnees = abs(ball_train_feet_dis)
                        print("Ball distance betweeen traning leg are ", ballDisBtKnees)
                        if ballDisBtKnees <= 0.15*kneesDis:
                            ball_release = "Delayed 2"
                        else:
                            ball_release = "Late"
                    elif ball_train_feet_dis > 0 and ball_slide_feet_dis < 0:
                        ballDisBtKnees = abs(ball_train_feet_dis)
                        print("Ball distance betweeen traning leg are ", ballDisBtKnees)
                        if ballDisBtKnees  <= 0.15*kneesDis:
                            ball_release = "Delayed 2"
                        elif ballDisBtKnees <= 0.85*kneesDis:
                            ball_release = "Delayed 1"
                        else:
                            ball_release = "Traditional"
                    elif ball_slide_feet_dis > 0:
                        ball_release = "Early"
                    currentframenumber['Release']=[currentFrame+1]
    
        # Append to array
        feetStuff = {"Frame": currentFrame+1, "LH_X":l_heel_x,"LI_X":l_ind_x,"Velocity": velo,"Wrist_Y":r_wrist_y,"Wrist_X":r_wrist_x}
        feetVelo = feetVelo.append(feetStuff, ignore_index=True)

        # Calculate torso and neck angles
        torso_inclination = findAngle(l_hip_x, l_hip_y, l_shldr_x, l_shldr_y)

        #============ Annotations onto video ============
        # Define font size 
        if h >= 2160 and font_access == 1:
            print("bigger than 2160", h)
            thick = 10
            text_y = 80
            fontsize = 3
            font_access = 0
        elif h >= 1080 and font_access == 1:
            print("bigger than 1080, ", h)
            fontsize = 1.5
            font_access = 0
        elif font_access == 1:
            print("smaller than 1080, ", h)
            thick = 2
            fontsize = 0.6
            font_access = 0

        # # BACK ANGLE
        # # Check for Camera Alignment to be in Proper Sideview
        # offset = findDistance(l_shldr_x, l_shldr_y, r_shldr_x, r_shldr_y)
        # if offset < 100:
        #     cv2.putText(image, str(int(offset)) + ' Aligned', (10, text_y), font, fontsize, green, thick)
        # else:
        #     cv2.putText(image, str(int(offset)) + ' Not Aligned', (10 , text_y), font, fontsize, red, thick)
        # # Draw landmarks
        # cv2.circle(image, (l_shldr_x, l_shldr_y), 7, yellow, -1)
        # cv2.circle(image, (l_shldr_x, l_shldr_y - 100), 7, yellow, -1)
        # # Right shoulder is pink ball
        # cv2.circle(image, (r_shldr_x, r_shldr_y), 7, pink, -1)
        # cv2.circle(image, (l_hip_x, l_hip_y), 7, yellow, -1)
        # cv2.circle(image, (l_hip_x, l_hip_y - 100), 7, yellow, -1)
        # # STR for back angle
        # angle_text_string ='Frame: '+str(currentFrame)+' Torso Angle : ' + str(int(torso_inclination)) + 'deg Feet distance: '+ str(int(feetDist)) + ' Steps: '+ str(int(steps)) 
        # cv2.putText(image, angle_text_string, (10, text_y), font, fontsize, dark_blue, thick, cv2.LINE_AA)
        # # Join landmarks
        # cv2.line(image, (l_shldr_x, l_shldr_y), (l_shldr_x, l_shldr_y - 100), green, thick)
        # cv2.line(image, (l_hip_x, l_hip_y), (l_shldr_x, l_shldr_y), green, thick)
        # cv2.line(image, (l_hip_x, l_hip_y), (l_hip_x, l_hip_y - 100), green, thick)
        # # Display angles on the annotation
        # cv2.putText(image, str(int(torso_inclination)), (l_hip_x + 10, l_hip_y), font, fontsize, pink, thick, cv2.LINE_AA)

        # TIMING
        # Display the skeleton
        mp_drawing.draw_landmarks(
            image,
            keypoints.pose_landmarks,
            mp_pose.POSE_CONNECTIONS,
            landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style())
        # Text for Neck/Torso Angle, Feet distance & Steps

        # STR for timing
        angle_text_string = 'Frame: '+str(currentFrame) +' Feet distance: '+ str(int(feetDist)) + ' Steps: '+ str(int(steps))  + ' Release: '+ str(ball_release) +" Thrown " +str(throwing)+' Velocity '+ str(velo) 
        cv2.putText(image, angle_text_string, (10, text_y), font, fontsize, dark_blue, thick, cv2.LINE_AA)


        # if torso_inclination >= 43:
        #     detectedFrame = cap.get(cv2.CAP_PROP_POS_FRAMES)
        #     currentframenumber.append(detectedFrame-3)
        # # Write frames.
        # video_output.write(image)

        # Write frames.
        video_output.write(image)
    else:
        # # print("Cannot Detect Anything")
        # # STR for timing
        # currentFrame = cap.get(cv2.CAP_PROP_POS_FRAMES)
        # feetDist = "-"
        # steps = "-"
        # ball_release = "-"
        # velo = "-"

        # angle_text_string = 'Frame: '+str(currentFrame) +' Feet distance: '+ feetDist + ' Steps: '+ steps  + ' Release: '+ ball_release + ' Velocity '+ velo
        # cv2.putText(image, angle_text_string, (10, text_y), font, fontsize, dark_blue, thick, cv2.LINE_AA)
        feetStuff = {"Frame": currentFrame+1, "LH_X":0,"LI_X":0,"Velocity": 0,"Wrist_Y":0,"Wrist_X":0}
        feetVelo = feetVelo.append(feetStuff, ignore_index=True)
        print("Cannot Detect Anything!")
        # Write frames.
        video_output.write(image)
print('Video is done!')
cap.release()
video_output.release()

Starting...
Cannot Detect Anything!
Cannot Detect Anything!
Cannot Detect Anything!
Cannot Detect Anything!
Cannot Detect Anything!
Cannot Detect Anything!
Cannot Detect Anything!
Cannot Detect Anything!
Cannot Detect Anything!
Cannot Detect Anything!
Cannot Detect Anything!
Cannot Detect Anything!
Cannot Detect Anything!
Cannot Detect Anything!
Cannot Detect Anything!
Cannot Detect Anything!
Cannot Detect Anything!
Cannot Detect Anything!
Cannot Detect Anything!
trying
bigger than 1080,  1080
trying
trying
trying
trying
trying
trying
trying
trying
trying
trying
trying
trying
trying
trying
trying
trying
trying
trying
trying
trying
trying
trying
trying
trying
trying
trying
trying
trying
trying
trying
trying
trying
trying
trying
trying
trying
trying
trying
trying
trying
trying
trying
trying
trying
trying
trying
trying
trying
trying
trying
trying
trying
trying
trying
trying
trying
trying
trying
trying
trying
trying
trying
trying
trying
trying
trying
trying
trying
trying
trying
trying
tryi

In [8]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(feetVelo)

     Frame    LH_X    LI_X  Heel Velo  Index Velo  Wrist_Y  Wrist_X  Velocity
0      2.0     0.0     0.0        NaN         NaN      0.0      0.0       0.0
1      3.0     0.0     0.0        NaN         NaN      0.0      0.0       0.0
2      4.0     0.0     0.0        NaN         NaN      0.0      0.0       0.0
3      5.0     0.0     0.0        NaN         NaN      0.0      0.0       0.0
4      6.0     0.0     0.0        NaN         NaN      0.0      0.0       0.0
5      7.0     0.0     0.0        NaN         NaN      0.0      0.0       0.0
6      8.0     0.0     0.0        NaN         NaN      0.0      0.0       0.0
7      9.0     0.0     0.0        NaN         NaN      0.0      0.0       0.0
8     10.0     0.0     0.0        NaN         NaN      0.0      0.0       0.0
9     11.0     0.0     0.0        NaN         NaN      0.0      0.0       0.0
10    12.0     0.0     0.0        NaN         NaN      0.0      0.0       0.0
11    13.0     0.0     0.0        NaN         NaN      0.0      

# Re-Reading Analysed Video for Screenshotting


In [9]:


# path ='./Analysedphoto'
# isExist = os.path.exists(path)

# if not isExist:
#   # Create a new directory because it does not exist 
#   os.makedirs(path)
#   print("Analysedphoto folder is created!")

# cap=cv2.VideoCapture('20220725_203806REAL.mp4')
# x=0
# bool=True
# testing=[210.0]
# frameLen=int(cap.get(cv2.CAP_PROP_FRAME_COUNT)) # total number of frames in the video
# while bool:
#     for i in range(0,frameLen,1):
#         if x>=len(testing):
#             bool=False
#             break
#         # print("iv1 ", i)
#         # print(x<len(currentframenumber))
#         ret, frame= cap.read()
#         cap.set(cv2.CAP_PROP_POS_FRAMES, i)
                
#         if i== testing[x]:
#             print(cap.get(cv2.CAP_PROP_POS_FRAMES))
#             # print("iv2", i)
#             x=x+1
#             print(x)
#             cv2.imwrite("Analysedphoto/frame%d.jpg" % i, frame)     # save frame as JPEG file      
#             print('Read a new frame: ', ret)

